# Source

> Source code for **y**et**a**nother**s**olve**i**t

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

/opt/conda/lib/python3.12/site-packages/nbdev/doclinks.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources,importlib


In [ ]:
#| export
from ipylab import JupyterFrontEnd, Panel
from ipywidgets import Button, Checkbox, Dropdown, Text, FloatSlider, IntSlider
import openai
import json
import asyncio

class JupyterChat:
    """Integrates a chatbot into JupyterLab, allowing users to interact with an OpenAI model directly within notebooks."""
    def __init__(self, 
                 api_key : str=None,  # api key for the openai api
                 openai_base_url : str =None, # base url of the openai api
                 model : str =None, # model id for the openai api
                 tag_system : str ='#| chat_system', # tag for system chat markdown cells
                 tag_user : str ='#| chat_user', # tag for user chat markdown cells
                 tag_assistant : str ='#| chat_assistant' # tag for assistant chat markdown cells
                ):
        self.client = openai.Client(base_url=openai_base_url, api_key=api_key)
        self.app = JupyterFrontEnd()
        self.get_models()
        self.model = model
        self.latest_response = None
        self.tag_system =  tag_system
        self.tag_user = tag_user
        self.tag_assistant = tag_assistant

        self.max_tokens = None
        self.temperature = None
        self.presence_penalty = None

        self.setup_ipylab_panel()
        self.setup_ipylab_toolbar()

    def get_models(self):
        """Create list of model objects from the Openai client"""
        available_models = self.client.models.list()
        self.models = [{"company" : model.name.split(':')[0] if model.name else model.id.split('/')[0],
                        "name" :  model.name if model.name else model.id.split('/')[-1], 
                        "id" : model.id,
                       }for model in available_models ]

    def setup_ipylab_panel(self):
        """Add an ipylab panel to browse and select available models"""
        companies = tuple(set([m['company'] for m in self.models]))
        company = 'Meta' if 'Meta' in companies else companies[0]
        filtered_models = tuple([m['id'] for m in self.models if m['company'] == company])
        my_fav_model = 'meta-llama/llama-3.1-8b-instruct'
        model = my_fav_model if my_fav_model in filtered_models else filtered_models[0]
        self.wgt_dd_company = Dropdown(options=companies, value=company, description='Company', tooltip='', layout={'width': '600px'})
        self.wgt_dd_company.observe(self.update_any_widget)
        self.wgt_dd_model = Dropdown(options=filtered_models, value=model, description='Model', tooltip='', layout={'width': '600px'})
        self.wgt_dd_model.observe(self.update_model_widget)
        self.wgt_txt_search_models = Text(value="", description='Search',disabled=False,indent=False)
        self.wgt_txt_search_models.observe(self.update_any_widget)
        self.wgt_btn_close_panel =  Button(button_style='danger',tooltip='Close Panel',icon='window-close ')
        self.wgt_btn_close_panel.on_click(self.remove_ipylab_features)

        # Openai Body parameters
        self.wgt_is_max_tokens = IntSlider(value=256, min=16, max=4096, description="Max tokens:", step=1)
        self.wgt_is_max_tokens.observe(self.update_any_widget)
        self.wgt_fs_temperature = FloatSlider(value=0.2, min=0.0, max=2.0, description="Temperature:", step=0.1, continuous_update=False)
        self.wgt_fs_temperature.observe(self.update_any_widget)
        self.wgt_fs_presence_penalty = FloatSlider(value=0.0, min=-2.0, max=2.0, description="Presence penalty:", step=0.1, continuous_update=False)
        self.wgt_fs_presence_penalty.observe(self.update_any_widget)
        
        panel = Panel()
        panel.children = [self.wgt_btn_close_panel, self.wgt_dd_company, self.wgt_dd_model, #self.wgt_txt_search_models,
                          self.wgt_is_max_tokens, self.wgt_fs_temperature, self.wgt_fs_presence_penalty]
        panel.title.label = 'Yasi'
        panel.title.icon_class = 'fa fa-robot'
        panel.title.closable = True
        self.panel = panel
        self.app.shell.add(self.panel, 'right', { 'rank': '1000'})

    def add_message_cell(self, id):
        content =f"{self.tag_assistant}\n\n" if id == "yasi-a" else f"{self.tag_user}\n\n"
        self.create_new_markdown_cell(content)

    def setup_ipylab_toolbar(self):
        self.app.commands.add_command("add_message_cell", self.add_message_cell)
        self.app.commands.add_command("send_yasi_dialog", self.send_dialog)
       
        self.app.toolbar.add_button("yasi-u", "add_message_cell", { "id" : "yasi-u" }, iconClass='fa fa-comment', tooltip="Yasi: Add user message", after = "cellType")
        self.app.toolbar.add_button("yasi-a", "add_message_cell", { "id" : "yasi-a" }, iconClass='fa fa-robot', tooltip="Yasi: Add assistant message", after = "yasi-u")
        self.app.toolbar.add_button("yasi-send", "send_yasi_dialog", { "id" : "yasi-send" }, iconClass='fa fa-paper-plane', tooltip="Yasi: Send dialog to AI", after = "yasi-a")

    def remove_ipylab_features(self, change):
        self.panel.close()
        self.app.toolbar.remove_button("yasi-u")
        self.app.toolbar.remove_button("yasi-a")
        self.app.toolbar.remove_button("yasi-send")

    def update_any_widget(self, change):
        """Update the options of the widgets based on the selection"""
        selected_company = self.wgt_dd_company.value
        search = self.wgt_txt_search_models.value
        companies = companies = tuple(set([m['company'] for m in self.models if (search in m['id'])]))
        model_ids = tuple([m['id'] for m in self.models if (m['company'] == selected_company) & (search in m['id'])])
        self.wgt_dd_model.options = model_ids

        self.max_tokens = self.wgt_is_max_tokens.value
        self.temperature = self.wgt_fs_temperature.value
        self.presence_penalty = self.wgt_fs_presence_penalty.value
    
    def update_model_widget(self, change):
        self.model = self.wgt_dd_model.value
    
    def create_new_markdown_cell(self, 
                                 content: str # Markdown content
                                ):
        """Adds a new markdown cell with the given content below"""
        self.app.commands.execute('notebook:insert-cell-below')
        self.app.commands.execute('notebook:replace-selection', { 'text': content})
        self.app.commands.execute('notebook:change-cell-to-markdown')

    def send_chat_completions(self, 
                              messages: list # A list of messages comprising the conversation so far.
                             ):
        """Send messages a.k.a dialog to the Openai chat completions API"""
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages= messages,
                max_tokens = self.max_tokens,
                temperature = self.temperature,
                presence_penalty = self.presence_penalty
            )
            self.latest_response = response
            response_text = response.choices[0].message.content.strip()
        except Exception as e: 
            response_text = f'There was an problem communicating with the Openai API:\n\n{e}'

        # Format the response with Markdown-like style in a code cell
        formatted_response = response_text.strip()

        # Insert a new cell below with the assistant's response
        self.create_new_markdown_cell(formatted_response)

    def send_query(self, user_prompt: str):
        """Send user prompt to chatbot and insert response as new cell"""
        self.send_chat_completions(messages=[{"role": "user", "content": user_prompt}])

    def get_current_nb(self):
        """Saves the the notebook returns the JSON content"""
        self.app.commands.execute('docmanager:save')
        fn = self.app.sessions.current_session['name']
        with open(fn, 'r') as f:
            current_notebook = json.load(f)
        return current_notebook
    
    def extract_notebook_dialog(self):
        """Extracts the tagged dialog form the current notebook, and turns it into a messages list"""
        current_notebook = self.get_current_nb()
        tmp_messages = []
        for cell in current_notebook['cells']:
            found_tags = {
                'system' : tag_in_cell(cell, self.tag_system),
                'user' : tag_in_cell(cell, self.tag_user),
                'assistant': tag_in_cell(cell, self.tag_assistant)
            }
            roles = [k for k,v in found_tags.items() if v == True]
            if len(roles) == 1:
                role = roles[0]
                tmp_content = ''.join(cell['source'])
                tmp_messages.append({"role": role, "content": tmp_content})
            if len(roles) > 1:
                tmp_content = '## ⚠⚠⚠ cell contains multiple tags ⚠⚠⚠\n'
                tmp_content += 'identify the following cell and select only one tag\n'
                tmp_content += '> ' + '> '.join(cell['source'])
                self.create_new_markdown_cell(tmp_content)
        return tmp_messages

    def send_dialog(self, id=None):
        """Sends the dialoge to the openai api and adds the response as a new cell below"""
        self.send_chat_completions(messages=self.extract_notebook_dialog())


In [ ]:
#| export
def tag_in_cell(cell, # Dictonary of a Jupyter Notebook cell
                tag # The tag to search
               )->bool: # True if any line contains the given tag
    """Checks a Jupyter Notebook cells source, if any line starts with the given tag"""
    return any([line.startswith(tag) for line in cell['source']])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()